In [53]:
import os
import pandas as pd
import numpy as np

root = "/home/arnau/PycharmProjects/MAEGG_IPPO/"

w_e = {
    0.2: {
        1: 2.139,
        10: 2.674,
        1000: 2.656
    },
    0.4: {
        1: 2.505,
        10: 1.98,
        1000: 10
    },
    0.6: {
        1: 7.842,
        10: 6.472,
        1000: 6.76
    },
    0.8: {
        1: 8.982,
        10: 6.678,
        1000: 2.556
    }
}

In [54]:
# build the whole statistics db

eff_rates = [0.2, 0.4, 0.6, 0.8]
dbs = [0, 1, 10, 1000]
header = None
data = {}

for eff_rate in eff_rates:
    for db in dbs:
        try:
            os.chdir(root)
            os.chdir(f"EGG_DATA/db{db}_effrate{eff_rate}_we{w_e[eff_rate][db]}_ECAI_new/db{db}_effrate{eff_rate}_we{w_e[eff_rate][db]}_ECAI_new")
            df = pd.read_csv("stats.csv")
            if header is None:
                header = list(df.columns)
            data[(eff_rate, db, w_e[eff_rate][db])] = df.to_numpy()
        except Exception as e:
            continue
            

In [55]:
header

['Agent',
 'Apples Stepped',
 'Apples Stepped Ratio',
 'Apples gathered',
 'Apples dropped',
 'Apples from box']

In [56]:
# View one register
get = lambda eff_rate, db : data[(eff_rate, db, w_e[eff_rate][db])]
get(0.2, 1)

array([[0.000e+00, 9.728e+01, 3.200e-01, 8.260e+01, 1.468e+01, 0.000e+00],
       [1.000e+00, 2.796e+01, 9.000e-02, 5.810e+00, 2.215e+01, 1.117e+01],
       [2.000e+00, 3.623e+01, 1.200e-01, 7.100e+00, 2.913e+01, 1.069e+01],
       [3.000e+00, 3.403e+01, 1.100e-01, 6.760e+00, 2.727e+01, 1.084e+01],
       [4.000e+00, 2.223e+01, 7.000e-02, 4.680e+00, 1.755e+01, 1.181e+01]])

In [222]:
# View of fixed db size, change eff rate and barplot the number of dropped apples for each eff rate
import matplotlib.pyplot as plt

def get_some_data(eff_rate):
    dbs = [1, 10, 1000]
    
    dropped_apples = []
    gathered_apples = []
    stepped_apples = []
    competition = []
    ineff_stepped = []
    eff_stepped = []
    generated_apples = []
    leftover_apples = []
    for db in dbs:
        dropped = get(eff_rate,db)[:, header.index('Apples dropped')]
        gathered = get(eff_rate,db)[:, header.index('Apples gathered')]
        stepped = get(eff_rate,db)[:, header.index('Apples Stepped')].sum()
        
        stepped_inefficients = get(eff_rate,db)[int(eff_rate*5):, header.index('Apples Stepped')].mean()
        stepped_efficients = get(eff_rate,db)[:int(eff_rate*5), header.index('Apples Stepped')].mean()
        competition.append(stepped_efficients / stepped_inefficients)
        ineff_stepped.append(stepped_inefficients)
        eff_stepped.append(stepped_efficients)
        
        gen = stepped /  get(eff_rate,db)[:, header.index('Apples Stepped Ratio')].sum()
        generated_apples.append(gen)
        leftover_apples.append((1-get(eff_rate,db)[:, header.index('Apples Stepped Ratio')].sum()) * gen)
        
        dropped_apples.append(dropped.sum())
        gathered_apples.append(gathered.sum())
        stepped_apples.append(dropped.sum() + gathered.sum())
    
    return dropped_apples, gathered_apples, stepped_apples, competition, ineff_stepped, eff_stepped, generated_apples, leftover_apples


In [230]:
eff_rates = [0.2, 0.4, 0.6, 0.8]
ineff_stepped_arr = np.zeros((len(eff_rates), 3))
eff_stepped_arr = np.zeros((len(eff_rates), 3))
competition_arr = np.zeros((len(eff_rates), 3))
generated_apples_arr = np.zeros((len(eff_rates), 3))
leftover_apples_arr = np.zeros((len(eff_rates), 3))
stepped_apples_arr = np.zeros((len(eff_rates), 3))
gathered_apples_arr = np.zeros((len(eff_rates), 3))
for i, eff_rate in enumerate(eff_rates):
    dropped_apples, gathered_apples, stepped_apples, competition, ineff_stepped, eff_stepped, generated_apples, leftover_apples = get_some_data(eff_rate)
    ineff_stepped_arr[i] = np.array(ineff_stepped)
    eff_stepped_arr[i] = np.array(eff_stepped)
    competition_arr[i] = np.array(competition)
    generated_apples_arr[i] = np.array(generated_apples)
    leftover_apples_arr[i] = np.array(leftover_apples)
    stepped_apples_arr[i] = np.array(stepped_apples)
    gathered_apples_arr[i] = np.array(gathered_apples)
pd.DataFrame(stepped_apples_arr, columns=['db1', 'db10', 'db1000'], index=eff_rates)

,db1,db10,db1000
0.2,217.73,181.97,181.27
0.4,249.25,186.15,244.39
0.6,219.60,224.26,266.53
0.8,266.86,233.45,268.00
